In [1]:
import os
os.environ["SQLITE_BUSY_TIMEOUT"] = "30000"  # 30 segundos

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale, scale
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader

In [3]:
data_path= "C:/Users/victo/OneDrive/Escritorio/Máster IA/Redes Neuronales y Deep Learning/Deep/Deep-project/drive-download-20231121T165626Z-001/HotelReservationsPreparedCleanAttributes.csv"
label_path = "C:/Users/victo/OneDrive/Escritorio/Máster IA/Redes Neuronales y Deep Learning/Deep/Deep-project/drive-download-20231121T165626Z-001/HotelReservationsOutput.csv"

In [4]:
features  = pd.read_csv(data_path)
labels = pd.read_csv(label_path)

In [5]:
from sklearn.model_selection import train_test_split
# Divide el conjunto de datos
features_train, features_test, labels_train, labels_test = train_test_split(features.values, labels.values, test_size=0.1, random_state=42)
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.1, random_state=42)

# Convierte los arrays a tensores de PyTorch
features_train = torch.tensor(features_train).float()
labels_train = torch.tensor(labels_train).float()
features_val = torch.tensor(features_val).float()
labels_val = torch.tensor(labels_val).float()
features_test = torch.tensor(features_test).float()
labels_test = torch.tensor(labels_test).float()

# Crea conjuntos de datos de PyTorch
train_dataset = TensorDataset(features_train, labels_train)
val_dataset = TensorDataset(features_val, labels_val)
test_dataset = TensorDataset(features_test, labels_test)

# Crea DataLoaders de PyTorch
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
import tensorflow as tf
import torch

# Verificar si CUDA está disponible
cuda_available = torch.cuda.is_available()
print(f"CUDA disponible: {cuda_available}")

# Enumerar dispositivos de GPU disponibles y mostrar sus nombres si CUDA está disponible
if cuda_available:
    print("Listado de dispositivos GPU disponibles:")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No se detectó una GPU. Asegúrate de que el acelerador de hardware esté configurado.")


CUDA disponible: True
Listado de dispositivos GPU disponibles:
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU


In [7]:
import torch.nn as nn

class PyTorchModel(nn.Module):
    def __init__(self, trial, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Definir la arquitectura de la red
        n_layers = trial.suggest_int('n_layers', 1, 50)
        for i in range(n_layers):
            num_units = trial.suggest_categorical(f'n_units_l{i}', [16, 32, 64, 128, 256, 512, 1024, 2048])
            activation = trial.suggest_categorical(f'activation_l{i}', ['ReLU', 'Tanh', 'ELU', 'SELU', 'LeakyReLU'])
            dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5)
            use_batch_norm = trial.suggest_categorical(f'batch_norm_l{i}', [True, False])
            l2_reg = trial.suggest_float(f'l2_reg_l{i}', 1e-5, 1e-1, log=True)

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)  # Añadir la tasa de regularización L2

            current_shape = num_units

            # Añadir Batch Normalization si se indica
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            
            # Añadir Dropout
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = trial.suggest_float(f'leakyrelu_alpha_l{i}', 0.01, 0.3)
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss



In [8]:
from torch.optim import Adam, SGD, RMSprop
from torch.utils.data import DataLoader

def build_model(params, input_shape, is_trial=True):
    # Crear una instancia del modelo con la arquitectura definida
    model = PyTorchModel(params, input_shape)

    if is_trial:
        # Configuración durante la optimización con Optuna
        optimizer_name = params.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
        lr = params.suggest_float('lr', 1e-5, 1e-1, log=True)
        if optimizer_name == 'Adam':
            beta1 = params.suggest_float('adam_beta1', 0.85, 0.99)
            beta2 = params.suggest_float('adam_beta2', 0.9, 0.999)
            optimizer = Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
        elif optimizer_name == 'SGD':
            momentum = params.suggest_float('sgd_momentum', 0.1, 0.9)
            optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
        elif optimizer_name == 'RMSprop':
            alpha = params.suggest_float('rmsprop_alpha', 0.85, 0.99)
            optimizer = RMSprop(model.parameters(), lr=lr, alpha=alpha)
    else:
        # Configuración con parámetros fijos
        optimizer_name = params['optimizer']
        if optimizer_name == 'Adam':
            optimizer = Adam(model.parameters(), lr=params['lr'], betas=(params['adam_beta1'], params['adam_beta2']))
        elif optimizer_name == 'SGD':
            optimizer = SGD(model.parameters(), lr=params['lr'], momentum=params.get('sgd_momentum', 0.9))
        elif optimizer_name == 'RMSprop':
            optimizer = RMSprop(model.parameters(), lr=params['lr'], alpha=params.get('rmsprop_alpha', 0.99))

    return model, optimizer



In [9]:
from tqdm import tqdm
import torch

def calculate_accuracy(outputs, labels):
    predicted = outputs.round()  # Redondear para obtener las predicciones binarias
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)

In [10]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import gc
from torch.utils.data import DataLoader

def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, file_name_base):
    plt.figure(figsize=(12, 4))

    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    plt.savefig(f"{file_name_base}_plot.png")
    plt.close()

In [11]:
import json

best_loss_file = "global_best_loss.json"

# Intenta cargar la mejor precisión global desde un archivo
try:
    with open(best_loss_file, "r") as file:
        global_best_loss = json.load(file)
except (FileNotFoundError, json.JSONDecodeError, TypeError, ValueError):
    global_best_loss = np.inf

best_model_path = "best_model_loss.pt"
plot_file_name_base = "best_model_loss"

print(f"Mejor loss global: {global_best_loss}")

Mejor loss global: 0.28562170123798375


## Train

In [12]:
import torch
import gc
from torch.utils.data import DataLoader

def objective(trial):
    global global_best_loss, best_model_path, plot_file_name_base
    # Verificar y configurar el uso de la GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando {'GPU' if device.type == 'cuda' else 'CPU'} para el Trial {trial.number}")

    model, optimizer = build_model(trial, input_shape) 
    model.to(device)

    # Parametrización del número de épocas y tamaño del batch
    epochs = trial.suggest_int('epochs', 10, 2000)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024, 2048])

    # Criterio de pérdida y DataLoader
    criterion = torch.nn.BCELoss()  # Ajustar según tu problema
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Configuración de Early Stopping y Learning Rate Scheduler
    early_stopping_patience = trial.suggest_int('early_stopping_patience', 5, 50)
    factor = trial.suggest_float("lr_reduce_factor", 0.1, 0.9)
    lr_patience = trial.suggest_int("lr_reduce_patience", 5, 50)
    min_lr = trial.suggest_float("min_lr", 1e-6, 1e-4, log=True)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=lr_patience, min_lr=min_lr, verbose=True)

    # Entrenamiento del modelo
    print(f"Entrenando Trial {trial.number}...")
    best_loss = np.inf # Inicializacion de la mejor perdida
    epochs_no_improve = 0
    # Inicializar listas para almacenar métricas
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
            train_total += labels.size(0)

        train_loss /= train_total
        train_accuracy = train_correct / train_total

        # Almacenar las métricas de entrenamiento
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Evaluación y Early Stopping
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item() * inputs.size(0)
                val_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
                val_total += labels.size(0)

        val_loss /= val_total
        val_accuracy = val_correct / val_total

        # Almacenar las métricas de validación
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        lr_scheduler.step(val_loss)

        # Comprobar si este modelo es el mejor hasta ahora
        if val_loss < best_loss:
            best_loss = val_loss
            if best_loss < global_best_loss:
                global_best_loss = best_loss
                # Guardar el nuevo valor de global_best_loss
                with open(best_loss_file, "w") as file:
                    json.dump(global_best_loss, file)
                with open("best_loss_accuracy.json", "w") as file:
                    json.dump(val_accuracy, file)
                plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, plot_file_name_base)  # Sobreescribir el gráfico
                torch.save(model.state_dict(), best_model_path)  # Sobreescribir el mejor modelo
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == early_stopping_patience:
            print("Early stopping")
            break

    print(f"Trial {trial.number} completado. Mejor pérdida en validación: {best_loss}")

    # Limpiar la memoria después de cada trial
    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

    # Devolver la pérdida en lugar de la precisión
    return best_loss


In [13]:
input_shape = features_train.shape[1]  # Número de variables explicativas

In [10]:
import optuna

storage = "sqlite:///example_copy.db"  # Ruta de la base de datos SQLite
study_name = "accuracy_gpu_study_3"  # Nuevo nombre del estudio

# Crear un nuevo estudio
study = optuna.create_study(study_name=study_name, direction='minimize', storage=storage, load_if_exists=True)

[I 2023-12-09 13:25:37,542] Using an existing study with name 'accuracy_gpu_study_3' instead of creating a new one.


In [11]:
# Número de ensayos en el estudio
num_trials = len(study.trials)
print(f"El estudio contiene {num_trials} ensayos.")

El estudio contiene 159 ensayos.


In [16]:
trials_restantes=50000
study.optimize(objective, n_trials=trials_restantes)

Usando GPU para el Trial 119
Entrenando Trial 119...


Epoch   257: reducing learning rate of group 0 to 2.9506e-02.


[I 2023-12-06 13:21:49,953] Trial 119 finished with value: 0.2913436805398687 and parameters: {'n_layers': 24, 'n_units_l0': 512, 'activation_l0': 'SELU', 'dropout_l0': 0.0915941140303668, 'batch_norm_l0': False, 'l2_reg_l0': 0.0007790882168789851, 'n_units_l1': 2048, 'activation_l1': 'LeakyReLU', 'dropout_l1': 0.3498948822017353, 'batch_norm_l1': False, 'l2_reg_l1': 0.023703167715166243, 'leakyrelu_alpha_l1': 0.03151321845186461, 'n_units_l2': 64, 'activation_l2': 'ReLU', 'dropout_l2': 0.27457632456848474, 'batch_norm_l2': True, 'l2_reg_l2': 0.004321740063262309, 'n_units_l3': 2048, 'activation_l3': 'ELU', 'dropout_l3': 0.03311597343050048, 'batch_norm_l3': False, 'l2_reg_l3': 0.0001394958711871764, 'n_units_l4': 1024, 'activation_l4': 'SELU', 'dropout_l4': 0.4519005688427607, 'batch_norm_l4': True, 'l2_reg_l4': 0.0051566006787762304, 'n_units_l5': 16, 'activation_l5': 'SELU', 'dropout_l5': 0.043196791373979616, 'batch_norm_l5': True, 'l2_reg_l5': 0.00018174660283256554, 'n_units_l6':

Early stopping
Trial 119 completado. Mejor pérdida en validación: 0.2913436805398687
Usando GPU para el Trial 120
Entrenando Trial 120...


Epoch    57: reducing learning rate of group 0 to 2.6854e-02.


...

Trial 156 completado. Mejor pérdida en validación: 0.30927057390373663
Usando GPU para el Trial 157
Entrenando Trial 157...


[I 2023-12-06 18:13:37,768] Trial 157 finished with value: 0.3031070903103348 and parameters: {'n_layers': 22, 'n_units_l0': 512, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.18322621038716091, 'batch_norm_l0': False, 'l2_reg_l0': 0.0002204283115368935, 'leakyrelu_alpha_l0': 0.09438946309764938, 'n_units_l1': 2048, 'activation_l1': 'LeakyReLU', 'dropout_l1': 0.48208503286764787, 'batch_norm_l1': True, 'l2_reg_l1': 0.041686123476439796, 'leakyrelu_alpha_l1': 0.07281387762938243, 'n_units_l2': 512, 'activation_l2': 'SELU', 'dropout_l2': 0.23539423713759972, 'batch_norm_l2': True, 'l2_reg_l2': 0.027643501584101, 'n_units_l3': 64, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.002455148403035612, 'batch_norm_l3': False, 'l2_reg_l3': 0.0004715910720934802, 'leakyrelu_alpha_l3': 0.23713115921150735, 'n_units_l4': 128, 'activation_l4': 'SELU', 'dropout_l4': 0.43322395849240897, 'batch_norm_l4': True, 'l2_reg_l4': 0.002067615988078631, 'n_units_l5': 16, 'activation_l5': 'LeakyReLU', 'dropout_l

Trial 157 completado. Mejor pérdida en validación: 0.3031070903103348
Usando GPU para el Trial 158


[W 2023-12-06 18:13:42,350] Trial 158 failed with parameters: {'n_layers': 26, 'n_units_l0': 512, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.14013593859122542, 'batch_norm_l0': False, 'l2_reg_l0': 0.0002743724857902773, 'leakyrelu_alpha_l0': 0.1102640943594278, 'n_units_l1': 512, 'activation_l1': 'LeakyReLU', 'dropout_l1': 0.4558837603816325, 'batch_norm_l1': True, 'l2_reg_l1': 0.06894184978658485, 'leakyrelu_alpha_l1': 0.052074926749961926, 'n_units_l2': 512, 'activation_l2': 'SELU', 'dropout_l2': 0.22768576080184152, 'batch_norm_l2': True, 'l2_reg_l2': 0.01372210081227098, 'n_units_l3': 64, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.030868378213573806, 'batch_norm_l3': False, 'l2_reg_l3': 0.00025228485048148756, 'leakyrelu_alpha_l3': 0.2593620114463663, 'n_units_l4': 128, 'activation_l4': 'SELU', 'dropout_l4': 0.45083229097853034, 'batch_norm_l4': True, 'l2_reg_l4': 0.0012514897889118245, 'n_units_l5': 16, 'activation_l5': 'LeakyReLU', 'dropout_l5': 0.06350795011282388, 'batch_

KeyboardInterrupt: 

In [ ]:
print('Número de pruebas finalizadas:', len(study.trials))
print('Mejor prueba:', study.best_trial.params)

Número de pruebas finalizadas: 119
Mejor prueba: {'activation_l0': 'ReLU', 'activation_l1': 'LeakyReLU', 'activation_l10': 'ReLU', 'activation_l11': 'ReLU', 'activation_l12': 'LeakyReLU', 'activation_l13': 'LeakyReLU', 'activation_l14': 'ELU', 'activation_l15': 'SELU', 'activation_l16': 'ReLU', 'activation_l17': 'LeakyReLU', 'activation_l18': 'Tanh', 'activation_l19': 'Tanh', 'activation_l2': 'ReLU', 'activation_l20': 'ReLU', 'activation_l21': 'LeakyReLU', 'activation_l22': 'Tanh', 'activation_l23': 'LeakyReLU', 'activation_l24': 'SELU', 'activation_l25': 'SELU', 'activation_l26': 'LeakyReLU', 'activation_l27': 'ELU', 'activation_l3': 'LeakyReLU', 'activation_l4': 'ReLU', 'activation_l5': 'SELU', 'activation_l6': 'LeakyReLU', 'activation_l7': 'SELU', 'activation_l8': 'Tanh', 'activation_l9': 'ReLU', 'batch_norm_l0': False, 'batch_norm_l1': False, 'batch_norm_l10': True, 'batch_norm_l11': True, 'batch_norm_l12': True, 'batch_norm_l13': True, 'batch_norm_l14': True, 'batch_norm_l15': Fal

In [ ]:
study.best_params

{'activation_l0': 'ReLU',
 'activation_l1': 'LeakyReLU',
 'activation_l10': 'ReLU',
 'activation_l11': 'ReLU',
 'activation_l12': 'LeakyReLU',
 'activation_l13': 'LeakyReLU',
 'activation_l14': 'ELU',
 'activation_l15': 'SELU',
 'activation_l16': 'ReLU',
 'activation_l17': 'LeakyReLU',
 'activation_l18': 'Tanh',
 'activation_l19': 'Tanh',
 'activation_l2': 'ReLU',
 'activation_l20': 'ReLU',
 'activation_l21': 'LeakyReLU',
 'activation_l22': 'Tanh',
 'activation_l23': 'LeakyReLU',
 'activation_l24': 'SELU',
 'activation_l25': 'SELU',
 'activation_l26': 'LeakyReLU',
 'activation_l27': 'ELU',
 'activation_l3': 'LeakyReLU',
 'activation_l4': 'ReLU',
 'activation_l5': 'SELU',
 'activation_l6': 'LeakyReLU',
 'activation_l7': 'SELU',
 'activation_l8': 'Tanh',
 'activation_l9': 'ReLU',
 'batch_norm_l0': False,
 'batch_norm_l1': False,
 'batch_norm_l10': True,
 'batch_norm_l11': True,
 'batch_norm_l12': True,
 'batch_norm_l13': True,
 'batch_norm_l14': True,
 'batch_norm_l15': False,
 'batch_no

## Test

In [ ]:
import torch.nn as nn

class PyTorchModel(nn.Module):
    def __init__(self, params, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Definir la arquitectura de la red usando los hiperparámetros fijos
        n_layers = params['n_layers']
        for i in range(n_layers):
            num_units = params[f'n_units_l{i}']
            activation = params[f'activation_l{i}']
            dropout_rate = params[f'dropout_l{i}']
            use_batch_norm = params[f'batch_norm_l{i}']
            l2_reg = params[f'l2_reg_l{i}']

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)  # Añadir la tasa de regularización L2

            current_shape = num_units

            # Añadir Batch Normalization si se indica
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            
            # Añadir Dropout
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = params.get(f'leakyrelu_alpha_l{i}', 0.01)  # Valor predeterminado si no se proporciona
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss

In [ ]:
import torch

# Cargar el estado del modelo
state_dict = torch.load("best_model.pt")

# Imprimir la estructura del modelo
print(state_dict.keys())


odict_keys(['layers.0.weight', 'layers.0.bias', 'layers.3.weight', 'layers.3.bias', 'layers.6.weight', 'layers.6.bias', 'layers.7.weight', 'layers.7.bias', 'layers.7.running_mean', 'layers.7.running_var', 'layers.7.num_batches_tracked', 'layers.10.weight', 'layers.10.bias', 'layers.11.weight', 'layers.11.bias', 'layers.11.running_mean', 'layers.11.running_var', 'layers.11.num_batches_tracked', 'output.weight', 'output.bias'])


In [ ]:
model = PyTorchModel(study.best_trial.params, input_shape)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()  # Poner el modelo en modo de evaluación

RuntimeError: Error(s) in loading state_dict for PyTorchModel:
	Missing key(s) in state_dict: "layers.14.weight", "layers.14.bias", "layers.15.weight", "layers.15.bias", "layers.15.running_mean", "layers.15.running_var", "layers.18.weight", "layers.18.bias". 
	size mismatch for layers.6.weight: copying a param with shape torch.Size([2048, 512]) from checkpoint, the shape in current model is torch.Size([1024, 512]).
	size mismatch for layers.6.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for layers.7.weight: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for layers.7.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for layers.7.running_mean: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is tor

In [ ]:
def evaluate_model(model, criterion, test_loader, device):
    model.eval()  # Poner el modelo en modo de evaluación
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item()  # Sumar la pérdida del lote
            predicted = outputs.round()  # Redondear las salidas para obtener predicciones binarias
            correct += (predicted == labels).sum().item()  # Sumar predicciones correctas
            total += labels.size(0)

    test_loss /= len(test_loader)
    accuracy = correct / total
    return test_loss, accuracy

# Suponiendo que tienes un DataLoader para el conjunto de prueba llamado 'test_loader'
test_loss, test_accuracy = evaluate_model(model, criterion, test_loader, device)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Asegurarse de que el modelo esté en modo de evaluación y moverlo al dispositivo adecuado
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Realizar las predicciones
with torch.no_grad():
    inputs = features_test.to(device)
    outputs = model(inputs)
    predictions = outputs.round().cpu().numpy()  # Redondear las predicciones y moverlas al CPU

# Calcular y visualizar la matriz de confusión
conf_matrix = confusion_matrix(labels_test.numpy(), predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()

# Calcular y mostrar otras métricas
accuracy = accuracy_score(labels_test.numpy(), predictions)
recall = recall_score(labels_test.numpy(), predictions)
precision = precision_score(labels_test.numpy(), predictions)
f1 = f1_score(labels_test.numpy(), predictions)
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")